In [1]:
from LSTM_GumbelSoftmax import *
from dataset import make_data_generator
import json
from tqdm import tqdm_notebook as tqdm
#from torchsummary import summary

In [2]:
folder = 'data/IMDB/'
data_name = folder+'data.json'
# validation_name = folder+'valid_seq.json'
# testdata_name = folder+'testdata_seq.json'
vocab_name = folder+'vocab.json'

In [3]:
num_epochs = 1
save_rate = 1 #how many epochs per modelsave
#continue_from = "trained/Model1" # if none, put None
continue_from = None
epsilon = 1e-8
validation_size = 10000
device = torch.device('cuda')
!mkdir -p trained

In [4]:
vocab = json.load(open(vocab_name, 'r'))
VOC_SIZE = len(vocab)
INPUT_MAX = 300
SUMM_MAX = 50
UNK = "[UNK]"
BOS = "[CLS]"
EOS = "[SEP]"
PAD = "[PAD]"

In [5]:
batch_size = 6

training_set, training_generator = make_data_generator(\
data_name, INPUT_MAX, SUMM_MAX, vocab[PAD], batch_size, cutoff=None, shuffle=True, num_workers=4)

# validation_set, validation_generator = make_data_generator(\
# validation_name, INPUT_MAX, OUTPUT_MAX, vocab[PAD], batch_size, cutoff=validation_size, shuffle=False, num_workers=4)

def data_gen_train():
    for src, label, tgt in training_generator:
        src = src.to(device)
        label = (label).long().to(device)
        tgt = tgt.to(device)
        b = Batch(src, tgt, vocab[PAD])
        b.label = label
        yield b

loading json
load json done.


In [6]:
import math
total_train = int(math.ceil(training_set.size / batch_size))
# total_valid = int(math.ceil(validation_set.size / batch_size))
# print(total_train, total_valid)

In [7]:
def init_param(model):
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

def make_big_bird(vocab, N=6, 
               d_model=512, d_ff=2048, h=8, dropout=0.1, emb_share=False, bert_share=False):
    "Helper: Construct a model from hyperparameters."
    
    vocab_sz = len(vocab)
    
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    
    src_emb = nn.Sequential(Embeddings(d_model, vocab_sz), c(position))
    if emb_share:        
        tgt_emb = src_emb
        bert_class_emb = src_emb
        bert_discr_emb = src_emb
    else:
        tgt_emb = nn.Sequential(Embeddings(d_model, vocab_sz), c(position))
        bert_class_emb = nn.Sequential(Embeddings(d_model, vocab_sz), c(position))
        bert_discr_emb = nn.Sequential(Embeddings(d_model, vocab_sz), c(position))
    
    
    bert_class = BERT(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        bert_class_emb,
        vocab[PAD]
    )
    
    if bert_share:
        bert_discr = bert_class
    else:
        bert_discr = BERT(
            Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
            bert_discr_emb,
            vocab[PAD]
        )
        
    translator = LSTM_Gumbel_Encoder_Decoder(
        hidden_dim=d_model, 
        emb_dim=d_model, 
        input_len=INPUT_MAX, 
        output_len=SUMM_MAX, 
        voc_size=vocab_sz, 
        critic_net=CriticNet(2*d_model),
        device=device,
        eps=1e-8
    )
    
    reconstructor = LSTM_Normal_Encoder_Decoder(
        hidden_dim=d_model, 
        emb_dim=d_model, 
        input_len=SUMM_MAX, 
        output_len=INPUT_MAX, 
        voc_size=vocab_sz, 
        pad_index=vocab[PAD],
        device = device,
        eps=1e-8
    )
#     translator = Translator(
#         Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
#         Decoder(DecoderLayer(d_model, c(attn), c(attn), 
#                              c(ff), dropout), N),
#         src_emb,
#         tgt_emb,
#         Generator(d_model, vocab_sz, device),
#         CriticNet(d_model)
#         )
    
#     classifier = Classifier(
#         bert_class,
#         out_class = 5
#         # criterion = BCE
#     )
#     reconstructor = Reconstructor(
#         Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
#         Decoder(DecoderLayer(d_model, c(attn), c(attn), 
#                              c(ff), dropout), N),
#         src_emb,
#         tgt_emb,
#         Generator(d_model, vocab_sz, device),
#         vocab[PAD]
#     )   
    discriminator = Discriminator(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        d_model,
        len(vocab),
        vocab[PAD]
    )


    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for m in [translator, reconstructor, discriminator]:
        init_param(m)
        
#     if(str(device) == 'cpu'):
#         savedmodel = torch.load("pretrained/Translator4", map_location=lambda storage, location: storage)
#     else:
#         savedmodel = torch.load("pretrained/Translator4")
#     translator.load_state_dict(savedmodel['model'], strict=False)
#     if(str(device) == 'cpu'):
#         savedmodel = torch.load("pretrained/Translator4", map_location=lambda storage, location: storage)
#     else:
#         savedmodel = torch.load("pretrained/Translator4")
#     reconstructor.load_state_dict(savedmodel['model'])#, strict=False)
            
    # creation of big bird
    model = BigBird(
        translator, discriminator, reconstructor , 
        vocab, gamma=0.99, clip_value=0.5, #for WGAN, useless if WGAN-GP is used 
        lr_G = 5e-5,
        lr_D = 1e-4,
        lr_R = 1e-5,
        LAMBDA = 10, # Gradient penalty lambda hyperparameter
        RL_scale = 10,
        device = device
    )

    return model


In [8]:
model = make_big_bird(vocab, N=4, d_model=256, d_ff=256, h=4, dropout=0.1, emb_share=True, bert_share=True)
#model.load("Nest/NewbornBird_LSTM_GumbelSoftmax")

In [9]:
vocab_inv = {a:b for b, a in vocab.items()}
def convert_ids_to_tokens(ids):
    return [vocab_inv[i] for i in ids]

In [10]:
#start = 1 if continue_from == None else (int(continue_from.split("Model")[-1])+1)
history = []

start = model.epoch

#from tensorboardX import SummaryWriter
#writer = SummaryWriter('mygraph')
writer = None
all_loss = []
all_reward = []

for epoch in range(start, num_epochs+1):
    print("Epoch", epoch)
    
    # training
    stats = Stats()
    model.train()
    
    rewards = []
    
    trange = tqdm(enumerate(data_gen_train()), total=total_train)
    for i, batch in trange:
        #expect src has [CLS] and [SEP]
        loss, score  = model.run_iter(batch.src, batch.src_mask, SUMM_MAX, batch.trg, writer, D_iters=5, verbose = 1)
        trange.set_postfix(
            **{'Rec_loss': '{:.3f}'.format(loss[0])},
            **{'G_loss': '{:.3f}'.format(loss[1])},
            **{'D_loss': '{:.3f}'.format(loss[2])},
            **{'real_score': '{:.3f}'.format(score[0])},
            **{'fake_score': '{:.3f}'.format(score[1])},
            **{'acc': '{:.3f}'.format(score[2])},
        )
        stats.update(sum(loss), 1, log=0)
        all_loss.append(loss)
    model.epoch += 1
    t_h = stats.history
    history.append(t_h)
    #writer.add_scalar('reward', np.mean(t_h), epoch)
    print("[info] epoch train loss:", np.mean(t_h))
#writer.close()  
#     try:
#         torch.save({'model':model.state_dict(), 'training_history':t_h, 'validation_loss':np.mean(v_h)}, 
#                    "trained/Model"+str(epoch))
#     except:
#         continue

Epoch 0


origin:
['[CLS]', 'an', 'overblown', 'melodrama', 'typical', 'of', 'its', 'period', '(', 'mid', '-', '1950s', ')', 'and', 'appropriate', 'matinée', 'food', '.', 'rock', 'hudson', ',', 'the', 'hulk', 'everyone', 'always', 'falls', 'in', 'love', 'with', ',', 'plays', 'his', 'usual', 'stereotype', 'role', ',', 'but', 'whereas', 'in', 'giant', ',', 'made', 'the', 'same', 'year', ',', 'when', 'his', 'material', 'and', 'co', '-', 'stars', '(', 'taylor', '&', 'dean', ')', 'were', 'above', 'average', ',', 'in', 'this', 'movie', 'he', 'is', 'just', 'not', 'good', 'enough', 'to', 'raise', 'the', 'calibre', 'beyond', 'a', 'mushy', 'tale', 'of', 'how', 'difficult', 'it', 'is', 'to', 'be', 'both', 'rich', 'and', 'happy', '.', 'the', 'self', '-', 'destructive', 'brother', 'and', 'sister', '(', 'robert', 'stack', ',', 'reeling', 'his', 'way', 'through', 'the', 'film', 'in', 'a', 'drunken', 'stupor', ',', 'and', 'dorothy', 'malone', ',', 'playing', 'a', 'vampish', 'poor', 'little', 'rich', 'girl', 'to

origin:
['[CLS]', 'i', 'first', 'heard', 'about', 'this', 'film', 'about', '20', 'years', 'ago', 'when', 'i', 'was', 'a', 'kid', 'in', 'grade', 'school', '(', '!', ')', ',', 'it', 'just', 'so', 'happened', 'that', 'i', 'was', 'thumbing', 'through', 'the', 'encyclopedias', 'in', 'the', 'classroom', 'one', 'day', ',', 'and', 'under', 'the', 'entry', 'for', 'movies', '(', 'or', 'cinema', ',', 'i', 'do', "n't", 'remember', ')', ',', 'were', 'several', 'stills', 'for', 'different', 'movies', 'from', 'mainstream', 'to', 'experimental', ',', 'and', 'one', 'of', 'them', 'shown', 'on', 'the', 'page', 'was', 'a', 'still', 'for', 'offon', '.', 'it', 'really', 'intrigued', 'me', ',', 'since', 'it', 'stood', 'out', 'the', 'most', 'on', 'the', 'page', '(', 'it', 'was', 'a', 'still', 'from', 'the', 'film', 'of', 'the', 'scene', 'with', 'the', 'eye', 'with', 'other', 'elements', 'superimposed', 'over', 'it', ')', '.about', '18', 'or', 'so', 'years', 'later', ',', 'the', 'public', 'library', 'here', 'w

origin:
['[CLS]', 'this', 'movie', 'should', 'not', 'be', 'watched', 'as', 'it', 'was', 'meant', 'to', 'be', 'a', 'flop', '.', 'ram', 'gopal', 'verma', 'first', 'wanted', 'to', 'make', 'this', 'a', 'remake', 'of', 'classic', 'bollywood', 'movie', '"', 'sholay', '"', ',', 'but', 'after', 'having', 'problems', 'with', 'the', 'original', 'makers', 'decided', 'to', 'go', 'ahead', 'with', 'the', 'project', 'and', '.', '.', '.', 'i', 'guess', 'leave', 'all', 'the', 'good', 'parts', 'of', 'the', 'movie', '(', 'acting', ',', 'script', ',', 'songs', ',', 'music', ',', 'comedy', ',', 'action', 'etc', ')', 'out', 'and', 'shoot', 'the', 'movie', 'just', 'because', 'he', 'already', 'happen', 'to', 'hire', 'the', 'crew', '.', 'waste', 'of', 'money', ',', 'waste', 'of', 'time', '.', 'after', 'making', 'movies', 'like', 'rangeela', ',', 'satya', ',', 'and', 'company', 'he', 'pulled', 'a', 'coppola', '(', 'godfather', ')', 'on', 'us', ';', 'what', 'were', 'you', 'thinking', 'rgv', '?', 'anyways', ',', 

origin:
['[CLS]', 'bill', 'paxton', 'stars', 'in', 'and', 'directs', 'this', 'highly', 'original', 'film', '.', 'having', 'watched', 'the', 'first', 'time', 'i', 'was', 'by', 'how', 'good', 'it', 'was', '.', 'the', 'reviews', 'i', 'had', 'heard', 'were', 'ok', '.', 'as', 'a', 'result', 'i', 'was', 'expecting', 'an', 'average', 'thriller', 'at', 'most', '.however', 'because', 'of', 'paxtons', 'excellent', 'directing', 'and', 'acting', 'the', 'film', 'is', 'well', 'worth', 'watching', ',', 'especially', 'if', 'you', 'are', 'a', 'horror', 'film', 'fanatic', '.', 'the', 'film', 'is', 'also', 'helped', 'by', 'the', 'plot', 'twists', 'which', 'keep', 'coming', 'until', 'the', 'closing', 'credits', '.', 'the', 'films', 'strongest', 'point', 'is', 'the', 'storyline', 'which', 'i', 'have', 'to', 'say', 'is', 'highly', 'original', 'and', 'is', 'like', 'i', 'have', 'ever', 'seen', 'before', '.', 'well', 'done', 'also', 'to', 'the', '2', 'young', 'leads', 'which', 'perfectly', 'convey', 'the', 'em

...

origin:
['[CLS]', 'i', "'", 'm', 'fan', 'of', 'art', ',', 'i', 'like', 'anything', 'about', 'art', ',', 'i', 'like', 'paintings', ',', 'sculptures', ',', 'etc', '.', 'this', 'movie', 'shows', 'it', ',', 'so', 'i', 'like', 'it', 'a', 'lot', ',', 'it', 'shows', 'how', 'a', 'woman', 'wants', 'to', 'paint', 'anything', 'about', 'art', ',', 'especially', 'naked', 'bodies', ',', 'but', 'she', 'ca', "n't", 'do', 'it', 'because', 'of', 'her', 'strict', 'family', '(', 'father', ')', ',', 'at', 'the', 'beginning', 'of', 'the', 'movie', 'she', 'painted', 'herself', 'naked', ',', 'but', 'she', 'wanted', 'a', 'man', 'for', 'her', 'paintings', ',', 'but', 'her', 'family', 'did', "n't", 'let', 'her', 'paint', 'naked', 'men', 'because', 'it', "'s", 'against', 'the', 'moral', '.', 'even', 'so', 'artemisia', 'could', 'paint', 'her', 'boyfriend', 'and', 'her', 'art', 'teacher', 'completely', 'naked', '.', 'she', 'falls', 'in', 'love', 'with', 'her', 'art', 'teacher', ',', 'and', 'it', 'seems', 'the', 'ar

origin:
['[CLS]', 'let', 'this', 'serve', 'as', 'a', 'warning', 'to', 'anyone', 'wishing', 'to', 'draw', 'attention', 'to', 'themselves', 'in', 'the', 'media', 'by', 'linking', 'their', 'name', 'to', 'that', 'of', 'a', 'well', '-', 'loved', 'and', 'well', '-', 'respected', ',', 'not', 'to', 'say', 'revered', 'author', ',', 'in', 'order', 'to', 'draw', 'attention', 'to', 'their', 'home', '-', 'movies', 'out', 'on', 'dvd', '.', 'hyped', 'to', 'the', 'skies', 'by', 'its', 'obviously', 'talentless', 'makers', ',', 'in', 'fact', 'lied', 'about', 'only', 'to', 'be', 'revealed', ',', 'finally', ',', 'as', 'ludicrously', 'inept', 'in', 'every', 'department', ',', 'the', 'fans', 'of', 'wells', 'and', 'of', 'his', 'book', 'have', 'been', 'after', 'the', 'blood', 'of', 'its', 'writer', '-', 'producer', '-', 'director', 'since', 'it', 'appeared', 'on', 'dvd', '.', 'many', 'good', 'points', 'have', 'been', 'made', 'by', 'the', 'other', 'comments', 'users', 'on', 'this', 'page', '.', 'particularly',

origin:
['[CLS]', 'if', 'you', 'were', 'brought', 'up', 'on', 'a', 'diet', 'of', 'gameshows', 'you', "'ll", 'understand', 'that', 'you', 'gradually', 'need', 'a', 'bigger', 'and', 'better', 'fix', '.', 'well', ',', 'in', 'the', 'world', 'of', 'the', 'running', 'man', ',', 'your', 'needs', 'will', 'be', 'sated', '.', 'for', 'in', 'this', 'game', 'show', ',', 'prisoners', 'compete', 'for', 'freedom', ',', 'and', 'the', 'ultimate', 'prize', '-', 'their', 'very', 'lives', '.', 'i', 'loved', 'this', 'film', '.', 'it', 'was', 'such', 'a', 'parody', 'on', 'the', 'mind', '-', 'numbing', 'tripe', 'that', 'we', 'watch', 'on', 'a', 'daily', 'basis', '.', 'it', 'is', "n't", 'one', 'of', 'schwarzenegger', "'s", 'best', 'performances', ',', 'but', 'on', 'the', 'whole', 'it', 'is', 'a', 'very', 'good', 'film', '.', 'the', 'underlying', 'idea', 'that', 'television', 'corporations', 'will', 'one', 'day', 'be', 'the', '"', 'real', '"', 'rulers', 'of', 'the', 'planet', 'is', 'very', 'believable', ',', 'a

origin:
['[CLS]', 'i', "'", 'm', 'overwhelmed', 'by', 'the', 'work', 'of', 'jim', 'carrey', '.', 'i', 'keep', 'on', 'getting', 'this', 'movie', 'stuck', 'in', 'my', 'head', '.', 'the', 'grinch', 'liking', 'martha', 'may', ',', 'cindy', 'lou', '(', 'who', "'s", 'very', 'annoying', ';', 'her', 'sweet', 'innocence', ')', 'who', 'tries', 'to', 'get', 'the', 'grinch', 'in', 'the', 'christmas', 'spirit', ',', 'the', 'childhood', 'of', 'the', 'grinch', '(', 'very', 'funny', '!', ')', ',', 'and', 'moreover', 'the', 'weak', 'obvious', 'ending', 'with', '-', 'christmas', 'is', "n't", 'all', 'about', 'presents', '.', 'i', 'have', 'to', 'say', ',', 'i', 'felt', 'stupid', 'walking', 'out', 'of', 'the', 'theater', 'with', 'a', 'bunch', 'of', 'babies', 'and', 'toddlers', 'laughing', 'and', 'so', 'forth', ',', 'but', 'this', 'movie', 'was', 'a', 'good', 'full', '-', 'lengthed', 'adaption', 'of', 'dr', '.', 'seuss', "'", 's', 'short', 'film', 'and', 'is', 'for', 'all', 'ages', '.', '[SEP]', '[PAD]', '[

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
print(all_loss)

In [ ]:
plt.plot(range(len(all_loss)), all_loss)
plt.show()

In [ ]:
#print(model.all_rewards)
#plt.plot(range(len(model.all_rewards)), model.all_rewards)

In [ ]:
#plt.plot(range(len(model.all_rewards)-1), [sum(model.all_rewards[:i])/i for i in range(1,len(model.all_rewards))])